# Feature engineering, xgboost
https://www.kaggle.com/dlarionov/feature-engineering-xgboost


-　DateNum作成 'date_block_num',→YYYYMMで代用
- 祝日フラグ
- month
- days
- アイテムカテゴリ（デフォルト)-> カテゴリCategory、ジャンルType
- 月の売り上げ
### lag01
- 月の売り上げラグ 1,2,3,6,12
- アイテムあたりの「週Or日」平均販売個数 1,2,3,6,12　　'date_item_avg_item_cnt_lag_1',
- ショップあたり「週Or日」平均販売個数ラグ1，2，3，6，12　　'date_shop_avg_item_cnt_lag_1',
- アイテムカテゴリあたり「週Or日」平均販売個数ラグ1,2,3,6,12'date_cat_avg_item_cnt_lag_1',
- カテゴリあたり「週Or日」平均販売個数ラグ1,2,3,6,12，'date_cat_avg_item_cnt_lag_1',
- ジャンル　あたり[月] 平均販売個数ラグ1,2,3,6,12
- アイテムカテゴリ/ショップあたり[月] 平均販売個数ラグ1,2,3,6,12
- カテゴリ/ショップあたり[月] 平均販売個数ラグ1,2,3,6,12
- ジャンル/ショップ　あたり[月] 平均販売個数ラグ1,2,3,6,12
- ASP　月当たり
### lag02
- 価格トレンド　過去6か月
- 店舗売上トレンド
- 価格下落率
- 最終販売日
- 初販売日
- ショップあたりの最終販売日
- ショップあたりの初販売日
    'delta_price_lag',


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
%matplotlib inline

In [2]:
category_names = pd.read_csv('category_names.csv')
category_names.columns = ['ItemCategoryId', 'ItemCategoryName']

item_categories = pd.read_csv('item_categories.csv')
item_categories.columns = ['ItemId', 'ItemCategoryId']

sales_history = pd.read_csv('sales_history.csv')
sales_history.columns = ['Date', 'StoreId', 'ItemId', 'ItemPrice', 'SoldNum']
sales_history['Date'] = pd.to_datetime(sales_history['Date'])

test = pd.read_csv('test.csv')
test.columns = ['index', 'ItemId', 'StoreId']
#item_categories['ItemCategoryId'].unique()
#category_names.query('ItemCategoryName.str.contains("")')
'''170品目のリスト'''
test_item = test['ItemId'].unique()
test_item = pd.DataFrame({'ItemId':test_item})

'''年月のリスト'''
yyyymm = pd.read_csv('./csv/yyyymm.csv')

'''170品目と月番号加工済みデータ '''
#target = pd.read_csv('./csv/sales_170_date_block_asp_rev.csv')

''' ラグ01（販売個数トレンド+ASP+Revenue58特徴量'''
#lag01 = pd.read_csv('./csv/target_lag01.csv')
'''最終データ'''
# pd.read_csv('./csv/matrix_final.csv')
#test_item
'''カテゴリ名と連結'''
ttest = pd.merge(test,item_categories, on='ItemId')
ttest = pd.merge(ttest, category_names,on='ItemCategoryId')
print('予測データの品目数: ',ttest['ItemId'].nunique(),'\n',
      '訓練データの品目数: ',sales_history['ItemId'].nunique(),'\n',
      '予測データのカテゴリ数: ',item_categories['ItemCategoryId'].nunique(),'\n',
      '訓練データのカテゴリ数: ',ttest['ItemCategoryId'].nunique(),'\n',
     '売上の総数: ',sales_history['SoldNum'].sum(), '\n',
     '')

予測データの品目数:  170 
 訓練データの品目数:  9040 
 予測データのカテゴリ数:  26 
 訓練データのカテゴリ数:  26 
 売上の総数:  1262869.0 
 


In [3]:
lag01

,date_block_num,YYYYMM,YYYY,MM,StoreId,ItemId,ItemCategoryId,Category,Type,SoldNum,...,date_shop_cat_avg_item_cnt_lag_1,date_shop_cat_avg_item_cnt_lag_2,date_shop_cat_avg_item_cnt_lag_3,date_shop_cat_avg_item_cnt_lag_6,date_shop_cat_avg_item_cnt_lag_12,date_shop_type_avg_item_cnt_lag_1,date_shop_type_avg_item_cnt_lag_2,date_shop_type_avg_item_cnt_lag_3,date_shop_type_avg_item_cnt_lag_6,date_shop_type_avg_item_cnt_lag_12
0,0,201801,2018,1,0,1000001,100,5,1,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,201801,2018,1,0,1000002,100,5,1,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,201801,2018,1,0,1000004,100,5,1,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,201801,2018,1,0,1000005,100,5,1,8.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,201801,2018,1,0,1000006,100,5,1,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25402,21,201910,2019,10,17,2900075,290,1,22,6.0,...,2.04,NaN,NaN,NaN,NaN,2.00,NaN,NaN,NaN,NaN
25403,21,201910,2019,10,17,3200001,320,4,10,1.0,...,NaN,2.0,1.5,NaN,2.0,NaN,1.667,2.334,NaN,1.0
25404,21,201910,2019,10,17,3200026,320,4,10,4.0,...,1.50,2.0,1.5,NaN,NaN,1.50,1.667,2.334,NaN,NaN
25405,21,201910,2019,10,17,3400001,340,4,9,1.0,...,1.50,NaN,1.5,NaN,2.0,1.25,NaN,1.000,NaN,3.0


In [4]:
def lag_feature(df,lags,col):
    tmp = df[['date_block_num','StoreId','ItemId',col]]
    for i in lags:
        shifted = tmp.copy()
        shifted.columns = ['date_block_num','StoreId','ItemId',col+'_lag_'+str(i)]
        shifted['date_block_num'] += i
        df = pd.merge(df,shifted, on=['date_block_num','StoreId','ItemId'],how='left')
    return df

# 価格トレンド

In [5]:
item_ave =  lag01.groupby('ItemId').agg({'ItemPrice':'mean'}).reset_index()
item_ave.astype(np.float16)
item_ave = item_ave.rename(columns={'ItemPrice':'AvePrice'})

In [6]:
matrix = pd.merge(lag01,item_ave,on='ItemId',how='left')


In [7]:
matrix = matrix.rename(columns={'ItemPrice':'MonthlyAvePrice'})

In [8]:
lags = [1,2,3,4,5,6]
matrix = lag_feature(matrix,lags,'MonthlyAvePrice')

In [9]:
matrix

,date_block_num,YYYYMM,YYYY,MM,StoreId,ItemId,ItemCategoryId,Category,Type,SoldNum,...,date_shop_type_avg_item_cnt_lag_3,date_shop_type_avg_item_cnt_lag_6,date_shop_type_avg_item_cnt_lag_12,AvePrice,MonthlyAvePrice_lag_1,MonthlyAvePrice_lag_2,MonthlyAvePrice_lag_3,MonthlyAvePrice_lag_4,MonthlyAvePrice_lag_5,MonthlyAvePrice_lag_6
0,0,201801,2018,1,0,1000001,100,5,1,6.0,...,NaN,NaN,NaN,363.369709,NaN,NaN,NaN,NaN,NaN,NaN
1,0,201801,2018,1,0,1000002,100,5,1,2.0,...,NaN,NaN,NaN,237.512887,NaN,NaN,NaN,NaN,NaN,NaN
2,0,201801,2018,1,0,1000004,100,5,1,1.0,...,NaN,NaN,NaN,221.491935,NaN,NaN,NaN,NaN,NaN,NaN
3,0,201801,2018,1,0,1000005,100,5,1,8.0,...,NaN,NaN,NaN,245.844538,NaN,NaN,NaN,NaN,NaN,NaN
4,0,201801,2018,1,0,1000006,100,5,1,2.0,...,NaN,NaN,NaN,244.258373,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25402,21,201910,2019,10,17,2900075,290,1,22,6.0,...,NaN,NaN,NaN,589.942029,590.0,NaN,NaN,NaN,NaN,NaN
25403,21,201910,2019,10,17,3200001,320,4,10,1.0,...,2.334,NaN,1.0,3922.566667,NaN,4230.0,4230.0,4400.0,4230.0,NaN
25404,21,201910,2019,10,17,3200026,320,4,10,4.0,...,2.334,NaN,NaN,750.848013,680.0,680.0,680.0,850.0,850.0,NaN
25405,21,201910,2019,10,17,3400001,340,4,9,1.0,...,1.000,NaN,3.0,861.948824,850.0,NaN,850.0,850.0,850.0,NaN


In [10]:
for i in lags:
    matrix['delta_price_lag_'+str(i)] = (matrix['MonthlyAvePrice_lag_'+str(i)] -matrix[
        'AvePrice'] )/ matrix['AvePrice']

In [12]:
def select_trend(row):
    if row.first_valid_index() is None:
        return None
    else:
        return row[row.first_valid_index()]
        for i in lags:
            if row['delta_price_lag'+ str(i)]:
                return row['delta_price_lag'+str(i)]
        return 0

matrix['delta_price_lag'] = matrix.apply(select_trend, axis=1)
matrix['delta_price_lag'] = matrix['delta_price_lag'].astype(np.float16)
matrix['delta_price_lag'].fillna(0,inplace=True)

In [13]:
matrix

,date_block_num,YYYYMM,YYYY,MM,StoreId,ItemId,ItemCategoryId,Category,Type,SoldNum,...,MonthlyAvePrice_lag_4,MonthlyAvePrice_lag_5,MonthlyAvePrice_lag_6,delta_price_lag_1,delta_price_lag_2,delta_price_lag_3,delta_price_lag_4,delta_price_lag_5,delta_price_lag_6,delta_price_lag
0,0,201801,2018,1,0,1000001,100,5,1,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1,0,201801,2018,1,0,1000002,100,5,1,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2,0,201801,2018,1,0,1000004,100,5,1,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
3,0,201801,2018,1,0,1000005,100,5,1,8.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
4,0,201801,2018,1,0,1000006,100,5,1,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25402,21,201910,2019,10,17,2900075,290,1,22,6.0,...,NaN,NaN,NaN,0.000098,NaN,NaN,NaN,NaN,NaN,21.0
25403,21,201910,2019,10,17,3200001,320,4,10,1.0,...,4400.0,4230.0,NaN,NaN,0.078376,0.078376,0.121715,0.078376,NaN,21.0
25404,21,201910,2019,10,17,3200026,320,4,10,4.0,...,850.0,850.0,NaN,-0.094357,-0.094357,-0.094357,0.132053,0.132053,NaN,21.0
25405,21,201910,2019,10,17,3400001,340,4,9,1.0,...,850.0,850.0,NaN,-0.013863,NaN,-0.013863,-0.013863,-0.013863,NaN,21.0


In [14]:
features_to_drop = ['AvePrice','MonthlyAvePrice']

In [16]:
for i in lags:
    features_to_drop += ['MonthlyAvePrice_lag_'+str(i)]
    features_to_drop += ['delta_price_lag_'+str(i)]

matrix.drop(features_to_drop, axis=1,inplace=True)

In [21]:
matrix

,date_block_num,YYYYMM,YYYY,MM,StoreId,ItemId,ItemCategoryId,Category,Type,SoldNum,...,date_shop_cat_avg_item_cnt_lag_2,date_shop_cat_avg_item_cnt_lag_3,date_shop_cat_avg_item_cnt_lag_6,date_shop_cat_avg_item_cnt_lag_12,date_shop_type_avg_item_cnt_lag_1,date_shop_type_avg_item_cnt_lag_2,date_shop_type_avg_item_cnt_lag_3,date_shop_type_avg_item_cnt_lag_6,date_shop_type_avg_item_cnt_lag_12,delta_price_lag
0,0,201801,2018,1,0,1000001,100,5,1,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1,0,201801,2018,1,0,1000002,100,5,1,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2,0,201801,2018,1,0,1000004,100,5,1,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
3,0,201801,2018,1,0,1000005,100,5,1,8.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
4,0,201801,2018,1,0,1000006,100,5,1,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25402,21,201910,2019,10,17,2900075,290,1,22,6.0,...,NaN,NaN,NaN,NaN,2.00,NaN,NaN,NaN,NaN,21.0
25403,21,201910,2019,10,17,3200001,320,4,10,1.0,...,2.0,1.5,NaN,2.0,NaN,1.667,2.334,NaN,1.0,21.0
25404,21,201910,2019,10,17,3200026,320,4,10,4.0,...,2.0,1.5,NaN,NaN,1.50,1.667,2.334,NaN,NaN,21.0
25405,21,201910,2019,10,17,3400001,340,4,9,1.0,...,NaN,1.5,NaN,2.0,1.25,NaN,1.000,NaN,3.0,21.0


# 店舗売上トレンド

In [22]:
group = lag01.groupby(['date_block_num','StoreId']).agg({'Revenue':'sum'})
group.columns = ['date_shop_revenue']
group.reset_index(inplace=True)

matrix = pd.merge(matrix,group, on =['date_block_num','StoreId'],how='left')
matrix['date_shop_revenue'] = matrix['date_shop_revenue'].astype(np.float32)

group = group.groupby(['StoreId']).agg({'date_shop_revenue':'mean'})
group.columns = ['shop_avg_revenue']
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=['StoreId'],how='left')
matrix['shop_avg_revenue'] = matrix['shop_avg_revenue'].astype(np.float32)

matrix['delta_revenue'] = (matrix['date_shop_revenue'] - matrix
                          ['shop_avg_revenue']) / matrix['shop_avg_revenue']
matrix['delta_revenue'] = matrix['delta_revenue'].astype(np.float16)

matrix = lag_feature(matrix, [1],'delta_revenue')

matrix.drop(['date_shop_revenue','shop_avg_revenue','delta_revenue'],axis=1,inplace=True)

# 最終販売月

In [32]:
cache = {}
matrix['item_shop_last_sale'] = -1
matrix['item_shop_last_sale'] = matrix['item_shop_last_sale'].astype(np.int8)
for idx,row in matrix.iterrows():
    key = str(row.ItemId)+' '+str(row.StoreId)
    if key not in cache:
        if row.SoldNum!=0:
            cache[key] = row.date_block_num
    else:
        last_date_block_num = cache[key]
        matrix.at[idx, 'item_shop_last_sale'] = row.date_block_num - last_date_block_num
        cache[key] = row.date_block_num
        

In [33]:
cache = {}
matrix['item_last_sale'] = -1
matrix['item_last_sale'] = matrix['item_last_sale'].astype(np.int8)
for idx,row in matrix.iterrows():
    key = row.ItemId
    if key not in cache:
        if row.SoldNum!=0:
            cache[key] = row.date_block_num
    else:
        last_date_block_num = cache[key]
        if row.date_block_num> last_date_block_num:
            matrix.at[idx, 'item_last_sale'] = row.date_block_num - last_date_block_num
            cache[key] = row.date_block_num
        

# 初販売月

In [34]:
matrix['item_shop_first_sale'] = matrix[
    'date_block_num'] - matrix.groupby(['ItemId','StoreId']
                                      )['date_block_num'].transform('min')
matrix['item_first_sale'] = matrix[
    'date_block_num'] - matrix.groupby('ItemId')['date_block_num'].transform('min')

In [35]:
matrix

,date_block_num,YYYYMM,YYYY,MM,StoreId,ItemId,ItemCategoryId,Category,Type,SoldNum,...,date_shop_type_avg_item_cnt_lag_2,date_shop_type_avg_item_cnt_lag_3,date_shop_type_avg_item_cnt_lag_6,date_shop_type_avg_item_cnt_lag_12,delta_price_lag,delta_revenue_lag_1,item_shop_last_sale,item_last_sale,item_shop_first_sale,item_first_sale
0,0,201801,2018,1,0,1000001,100,5,1,6.0,...,NaN,NaN,NaN,NaN,0.0,NaN,-1,-1,0,0
1,0,201801,2018,1,0,1000002,100,5,1,2.0,...,NaN,NaN,NaN,NaN,0.0,NaN,-1,-1,0,0
2,0,201801,2018,1,0,1000004,100,5,1,1.0,...,NaN,NaN,NaN,NaN,0.0,NaN,-1,-1,0,0
3,0,201801,2018,1,0,1000005,100,5,1,8.0,...,NaN,NaN,NaN,NaN,0.0,NaN,-1,-1,0,0
4,0,201801,2018,1,0,1000006,100,5,1,2.0,...,NaN,NaN,NaN,NaN,0.0,NaN,-1,-1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25402,21,201910,2019,10,17,2900075,290,1,22,6.0,...,NaN,NaN,NaN,NaN,21.0,0.104248,1,-1,1,1
25403,21,201910,2019,10,17,3200001,320,4,10,1.0,...,1.667,2.334,NaN,1.0,21.0,NaN,2,-1,20,21
25404,21,201910,2019,10,17,3200026,320,4,10,4.0,...,1.667,2.334,NaN,NaN,21.0,0.104248,1,-1,8,9
25405,21,201910,2019,10,17,3400001,340,4,9,1.0,...,NaN,1.000,NaN,3.0,21.0,0.104248,1,-1,19,21


In [36]:
#matrix.to_csv('./csv/target_matrix.csv')

## 最初の12か月のデータはラグの値として計算しているので削除。いいかえればTestデータでは計算されないデータセット

In [45]:
matrix = matrix[matrix.date_block_num > 11]

In [46]:
matrix

,date_block_num,YYYYMM,YYYY,MM,StoreId,ItemId,ItemCategoryId,Category,Type,SoldNum,...,date_shop_type_avg_item_cnt_lag_2,date_shop_type_avg_item_cnt_lag_3,date_shop_type_avg_item_cnt_lag_6,date_shop_type_avg_item_cnt_lag_12,delta_price_lag,delta_revenue_lag_1,item_shop_last_sale,item_last_sale,item_shop_first_sale,item_first_sale
12690,12,201901,2019,1,0,1000001,100,5,1,2.0,...,1.250,NaN,1.667,3.125,12.0,NaN,2,1,12,12
12691,12,201901,2019,1,0,1000005,100,5,1,1.0,...,1.250,1.571,1.667,3.125,12.0,1.023438,1,1,12,12
12692,12,201901,2019,1,0,1000006,100,5,1,1.0,...,NaN,1.571,1.667,3.125,12.0,1.023438,1,1,12,12
12693,12,201901,2019,1,0,1000008,100,5,1,1.0,...,NaN,1.571,1.667,3.125,12.0,1.023438,1,1,12,12
12694,12,201901,2019,1,0,1000010,100,5,1,3.0,...,1.250,1.571,1.667,NaN,12.0,1.023438,1,1,10,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25402,21,201910,2019,10,17,2900075,290,1,22,6.0,...,NaN,NaN,NaN,NaN,21.0,0.104248,1,-1,1,1
25403,21,201910,2019,10,17,3200001,320,4,10,1.0,...,1.667,2.334,NaN,1.000,21.0,NaN,2,-1,20,21
25404,21,201910,2019,10,17,3200026,320,4,10,4.0,...,1.667,2.334,NaN,NaN,21.0,0.104248,1,-1,8,9
25405,21,201910,2019,10,17,3400001,340,4,9,1.0,...,NaN,1.000,NaN,3.000,21.0,0.104248,1,-1,19,21


# ラグの欠損値穴埋め

In [47]:
def fill_na(df):
    for col in df.columns:
        if ('_lag_' in col) & (df[col].isnull().any()):
            if ('item_cnt' in col):
                df[col].fillna(0,inplace=True)
    return df
matrix = fill_na(matrix)

C:\Users\kmtk49\Anaconda3\envs\signate_Ai_Quest_submission\lib\site-packages\pandas\core\series.py:4523: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


In [48]:
matrix

,date_block_num,YYYYMM,YYYY,MM,StoreId,ItemId,ItemCategoryId,Category,Type,SoldNum,...,date_shop_type_avg_item_cnt_lag_2,date_shop_type_avg_item_cnt_lag_3,date_shop_type_avg_item_cnt_lag_6,date_shop_type_avg_item_cnt_lag_12,delta_price_lag,delta_revenue_lag_1,item_shop_last_sale,item_last_sale,item_shop_first_sale,item_first_sale
12690,12,201901,2019,1,0,1000001,100,5,1,2.0,...,1.250,0.000,1.667,3.125,12.0,NaN,2,1,12,12
12691,12,201901,2019,1,0,1000005,100,5,1,1.0,...,1.250,1.571,1.667,3.125,12.0,1.023438,1,1,12,12
12692,12,201901,2019,1,0,1000006,100,5,1,1.0,...,0.000,1.571,1.667,3.125,12.0,1.023438,1,1,12,12
12693,12,201901,2019,1,0,1000008,100,5,1,1.0,...,0.000,1.571,1.667,3.125,12.0,1.023438,1,1,12,12
12694,12,201901,2019,1,0,1000010,100,5,1,3.0,...,1.250,1.571,1.667,0.000,12.0,1.023438,1,1,10,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25402,21,201910,2019,10,17,2900075,290,1,22,6.0,...,0.000,0.000,0.000,0.000,21.0,0.104248,1,-1,1,1
25403,21,201910,2019,10,17,3200001,320,4,10,1.0,...,1.667,2.334,0.000,1.000,21.0,NaN,2,-1,20,21
25404,21,201910,2019,10,17,3200026,320,4,10,4.0,...,1.667,2.334,0.000,0.000,21.0,0.104248,1,-1,8,9
25405,21,201910,2019,10,17,3400001,340,4,9,1.0,...,0.000,1.000,0.000,3.000,21.0,0.104248,1,-1,19,21


In [49]:
matrix.to_csv('./csv/matrix_final.csv')